# https://github.com/kernoelpanic/smartenv

# Smart Contract Crash Course - Basic interaction with `geth`

The most common client lib to interface with ethereum clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)
    
For the tutorial we will only interact with `geth` which uses his own local development testnet blockchain. 
* [geth console](https://geth.ethereum.org/docs/interface/javascript-console) has a link to the supported API via the geth console, which should be the "full" `web3.js` API.
* [geth cli options](https://geth.ethereum.org/docs/interface/command-line-options) only needed when running geth from the command line

## Connect via RPC/HTTP

Note that in our case `geth` is running in a private PoA chain setup.
Therefore, some configuration parameters are different. 

https://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority

For the challenge environment configuration the docker container for the geth client `Bob` is located at `172.18.0.8`.

In [44]:
import web3
from web3.middleware import geth_poa_middleware

w3_geth = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.8:8545"))
# check if connection was successful
assert w3_geth.isConnected()

w3_geth.middleware_onion.inject(geth_poa_middleware, layer=0)

In [45]:
# display the client version of the node we are connected to
w3_geth.clientVersion

'Geth/bob/v1.10.4-unstable-0e9c7d56-20210607/linux-amd64/go1.15.4'

In [46]:
# display network ID of client you are connected to
w3_geth.net.version

'1337'

In [47]:
# Display number of connected peers should be 1
w3_geth.net.peerCount

1

In [48]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3_geth.eth.syncing

False

In [49]:
w3 = w3_geth

## Blocks

In [16]:
# return current blockchain head of node
w3.eth.blockNumber

16611

In [22]:
# should be greater
!sleep 16
w3.eth.blockNumber

16635

In [18]:
# block 0 ist the genesis block of this testnet blockchain
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000033f4f5ac17d677e188ab8d43149717632f9960d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0xa5f0a078d24ba19a4f4f01b77aa1d13d965b95dfbdca6ba2fa79e711d481272e'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

As you see, the genesis block is special as it is the only block with no parent hash:

In [19]:
genesis_block['number']

0

In [20]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

The [genesis.json](../genesis_config/go-ethereum/berlin/genesis.json) file shows the node(s) that are allowed to mine blocks and the accounts that have been seeded with coins. 
The client we are currently connected to is not part of the PoA nodes that are allowed to create new blocks:

In [30]:
w3.eth.mining

False

## Accounts and balances

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

For the exercise you will recieve your personal account keypair which you have to copy in your `geth` folder to import it. 

In [33]:
ls /smartcode/datadir/bob/keystore

UTC--2021-06-07T13-20-25.685636254Z--a1273f73c607bd0af4d2916f4c9e6a550581dca6*


In [34]:
w3.eth.accounts

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6']

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase`. It is good practise to specifically assing a `default_account` some functions might cause problems if this is not explicitly assigned.

In [36]:
assert w3.eth.accounts[0] == w3.eth.coinbase
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account 

'0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6'

In [38]:
def getBalance(address):
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [39]:
getBalance(w3.eth.accounts[0]) 

Decimal('1000000')

### Create New Account

Create a new account with no password:

In [56]:
!geth --maxpeers 0 --datadir=/smartcode/datadir/bob/ --password=/dev/null account new

INFO [06-12|19:20:39.250] Maximum peer count                       ETH=0 LES=0 total=0
INFO [06-12|19:20:39.250] Smartcard socket not found, disabling    err="stat /run/pcscd/pcscd.comm: no such file or directory"

Your new key was generated

Public address of the key:   0x01A8719E91D8c0Fbecb28d41DA5F64e6427A7d94
Path of the secret key file: /smartcode/datadir/bob/keystore/UTC--2021-06-12T17-20-39.250731352Z--01a8719e91d8c0fbecb28d41da5f64e6427a7d94

- You can share your public address with anyone. Others need it to interact with you.
- You must NEVER share the secret key with anyone! The key controls access to your funds!
- You must BACKUP your key file! Without the key, it's impossible to access account funds!
- You must REMEMBER your password! Without the password, it's impossible to decrypt the key!



In [57]:
w3.eth.accounts

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 '0x01A8719E91D8c0Fbecb28d41DA5F64e6427A7d94']

## Payment Transaction and Gas

**Note:** When connected to `geth` or `parity` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `ganache` this is not necessary since it is a development environment.

Per default unlocking an account via HTTP is not allowed in `geth` so this has to be explicitly activated 
with `--allow-insecure-unlock` when starting `geth`. Do not use this flag in production! 

In [51]:
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"notforproductiveuse",0)

True

In [58]:
blkNumber_before = w3.eth.blockNumber
blkNumber_before

22829

In [59]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0x98f71e5a69ea90bc9e572a8bd8204a401a42567566e96fc20a68b79a551ea70e')

In [62]:
!sleep 16
blkNumber_after = w3.eth.blockNumber
assert blkNumber_before < blkNumber_after
blkNumber_after

22831

In [63]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
getBalance(w3.eth.accounts[0])

Decimal('999998.999979')

In [64]:
getBalance(w3.eth.accounts[1])

Decimal('1')

In [66]:
# get first (and only) transaction in the previously mined block
tx = w3.eth.getTransactionByBlock(blkNumber_before + 1,0)
tx 

AttributeDict({'blockHash': HexBytes('0x9e843f38d53c2c9c9c066b2323aca07d3269a05d0b1a833568857fdd92d8bf48'),
 'blockNumber': 22830,
 'from': '0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 'gas': 121000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0x98f71e5a69ea90bc9e572a8bd8204a401a42567566e96fc20a68b79a551ea70e'),
 'input': '0x',
 'nonce': 0,
 'to': '0x01A8719E91D8c0Fbecb28d41DA5F64e6427A7d94',
 'transactionIndex': 0,
 'value': 1000000000000000000,
 'type': '0x0',
 'v': 2710,
 'r': HexBytes('0xa300df88d7a17fdaef76f77e8d728bc5e137517898ca52cebe87e8eff4bab3ab'),
 's': HexBytes('0x5a237ebd7080004f60e4193223adc41c0afa2aa30b36165f048a9b07ba20f7a7')})

In [67]:
# get block and see the transaction id and gas used below
blk = w3.eth.getBlock(blkNumber_before + 1)
blk

AttributeDict({'difficulty': 2,
 'proofOfAuthorityData': HexBytes('0xd883010a03846765746888676f312e31352e34856c696e7578000000000000002c1bf698989bef982e67932dd8cb3defc227c8173ad442e4eaf17caa838c8b3e3e04ec3164ed39ac1062585d909ec214c051bd6be10a8f956cbdc340546809aa01'),
 'gasLimit': 8000000,
 'gasUsed': 21000,
 'hash': HexBytes('0x9e843f38d53c2c9c9c066b2323aca07d3269a05d0b1a833568857fdd92d8bf48'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x0000000000000000000000000000000000000000',
 'mix

In [69]:
# get the transaction ID of the first (and only) transaction in the block
blk["transactions"][0].hex()

'0x98f71e5a69ea90bc9e572a8bd8204a401a42567566e96fc20a68b79a551ea70e'

In [70]:
tx["hash"].hex() # hash of the transaction

'0x98f71e5a69ea90bc9e572a8bd8204a401a42567566e96fc20a68b79a551ea70e'

In [71]:
# Query the transaction by ID, also possible
w3.eth.getTransaction(blk["transactions"][0].hex())

AttributeDict({'blockHash': HexBytes('0x9e843f38d53c2c9c9c066b2323aca07d3269a05d0b1a833568857fdd92d8bf48'),
 'blockNumber': 22830,
 'from': '0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 'gas': 121000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0x98f71e5a69ea90bc9e572a8bd8204a401a42567566e96fc20a68b79a551ea70e'),
 'input': '0x',
 'nonce': 0,
 'to': '0x01A8719E91D8c0Fbecb28d41DA5F64e6427A7d94',
 'transactionIndex': 0,
 'value': 1000000000000000000,
 'type': '0x0',
 'v': 2710,
 'r': HexBytes('0xa300df88d7a17fdaef76f77e8d728bc5e137517898ca52cebe87e8eff4bab3ab'),
 's': HexBytes('0x5a237ebd7080004f60e4193223adc41c0afa2aa30b36165f048a9b07ba20f7a7')})

In [83]:
gasUsed = blk["gasUsed"] # gas used within block
gasUsed

21000

In [84]:
# estimate the gas price of an transaction without executing it
w3.eth.estimateGas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

The estimate yields the same result as the actual gas cost. 
Can this be different?

In [85]:
gasPrice = tx["gasPrice"] # The actual gas price attached to the tx
gasPrice

1000000000

In [86]:
# conversion between gas and ether specified per transaction
# i.e., how much wei a transaction is willing to pay for 
assert gasPrice == w3.eth.gasPrice # default value

In [89]:
# gasUsed(by the transaction) * gasPrice(specified in tx)
# gives the wei it costs to send it 
# times 10^18 gives the ether
tx_costs = (gasUsed * gasPrice) * 10**-18
tx_costs 

2.1000000000000002e-05

This is exaclty the missing cost on the sender side: 

In [90]:
float(getBalance(w3.eth.accounts[0]))

999998.999979

In [91]:
float(getBalance(w3.eth.accounts[0])) + tx_costs

999999.0

# TODO